In [3]:
import os
baseDirName = "Lab00"
print(os.path.abspath('.'))

/mnt/c/Users/user/Documents/Lab00/lib


In [1]:
# 平均絶対パーセント誤差 (MAPE)(Mean Absolute Percent Error (MAPE))を返す関数
# 引数として長さの同じ二つのリストをとる
# 引数l1: 実測値のリスト
# 引数l2: 予測値のリスト
# 単位：％

def returnMapeScore(l1, l2):
    return_num = 0
    if(len(l1) != len(l2)):
        print("引数のリストの長さが異なります", end=", ")
        return -1
    for i in range(len(l1)):
        l1_num = l1[i]
        l2_num = l2[i]

        return_num += abs((l1_num - l2_num)/l1_num)

    return_num /= len(l1)
    return_num *= 100
    return return_num


# 使用例：returnMapeScore([1,2,3,4], [4,3,2,1])
type(returnMapeScore([1, 2, 3, 4], [4, 3, 2, 1]))

def test_returnMapeScore():
    l1 = [1,2,3,4]
    l2 = [4,3,2,1]
    ansByFunc = returnMapeScore(l1, l2)
    ansByHand = (abs(1-4)/1 + abs(2-3)/2 + abs(3-2)/3 + abs(4-1)/4)/4 * 100
    # 多少の誤差を許容する
    ansByFunc = int(ansByFunc * 100) / 100
    ansByHand = int(ansByHand * 100) / 100
    
    assert ansByFunc == ansByHand